In [44]:
#%load_ext autoreload
# %autoreload 2

In [45]:
from state import get_state, queue
from basic.reward_basic import calculate_reward
from basic.learner_basic import Learner
from traffic_lights_maps import get_traffic_light_phases, make_map
import os
import sys

import traci
import random
import gym
from gym import spaces

In [46]:
from tensorflow.keras import Sequential, layers
from tensorflow.random import set_seed
import numpy as np
from collections import deque
from tensorflow import keras
from tensorflow import reduce_sum, reduce_mean, one_hot, GradientTape
from tensorflow.keras.losses import MeanSquaredError
import matplotlib.pyplot as plt

In [47]:
set_seed(42)  # extra code – ensures reproducibility on the CPU

input_shape = [36]  # == env.observation_space.shape
n_outputs = 4  # == env.action_space.n
#[((Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)), [0, 2, 4, 6])]
model_action = Sequential([
    layers.Dense(32, activation="elu", input_shape=input_shape),
    layers.Dense(32, activation="elu", input_shape=input_shape),
    # layers.Dense(2, activation="elu"),
    layers.Dense(n_outputs)])

In [48]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)  # random action
    else:
        Q_values = model_action.predict(state[np.newaxis], verbose=0)[0]
        print(f"state : {state}")
        print(f"Q_values : {Q_values}")
        return Q_values.argmax()  # optimal action according to the DQN

In [49]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    states, actions, rewards, next_states = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(4)
    ]
    return states, actions, rewards, next_states

In [50]:
batch_size = 32
discount_factor = 0.5
optimizer = keras.optimizers.Nadam(learning_rate=1e-1)
loss_fn = MeanSquaredError()

def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states = experiences  # a changer
    next_Q_values = model_action.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    # runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = one_hot(actions, n_outputs)
    with GradientTape() as tape:
        all_Q_values = model_action(states)
        Q_values = reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model_action.trainable_variables)
    optimizer.apply_gradients(zip(grads, model_action.trainable_variables))

In [59]:
sumoBinary = "/Users/arseneclaustre/sumo/bin/sumo-gui"
sumoBinary2 = "/Users/arseneclaustre/sumo/bin/sumo"
sumoConfig = r"C:/Program Files/rl_project/2025-02-12-16-24-37/osm.sumocfg"
sumoConfig2 = r"C:/Program Files/rl_project/2025-02-25-19-05-44/osm.sumocfg"
# sumoConfig3 = r"/home/psels/code/psels/RL_traffic/RL_traffic/Traditional_traffic/traditional_traffic.sumo.cfg"
sumoConfig3 = r"Traditional_traffic/traditional_traffic.sumo.cfg"


rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
replay_buffer = deque(maxlen=2000)


sumoCmd = [sumoBinary2, "-c", sumoConfig3, "--start"]
for episode in range(300):
    if traci.isLoaded():
        traci.close()
    traci.start(sumoCmd)
    lane_ids =  traci.lane.getIDList()
    # print(lane_ids[0])

    trafic_light_ids = traci.trafficlight.getIDList()

    state = np.array(queue(lane_ids))
    action=-1
    # print(state)
    wait_times.append(0)
    for step in range(10000): ## TO CHANGED
        epsilon = max(1 - episode / 300, 0.01)

        if step%1000 == 0:
            #print(f"longeur du buffer :{len(replay_buffer)}")
            ########################################################################
            ##Calcul de la reward
            next_state = np.array(queue(lane_ids))
            # reward = calculate_reward(values, reward, total_reward)[0]
            reward=(np.sum(state)-np.sum(next_state))
            replay_buffer.append((state, action, reward, next_state))

            list_values.append(queue(lane_ids))
            #########################################################################
            state=next_state
            action = epsilon_greedy_policy(state, epsilon)
            #print("action", action)
            traci.trafficlight.setPhase(trafic_light_ids[0],2*action)

            if len(replay_buffer) >= batch_size*10:
                training_step(batch_size)
                new_weights = model_action.get_weights()
                weights.append(new_weights)
                #print(f"Episode {episode}: new weights = {new_weights}")
            # else:
            #     print(f"Episode {episode}: pas assez de données dans le replay buffer.")
            # print(values)
            # if list_values:
            #    # print(f'list values {list_values[-1]}')


        traci.simulationStep()
    print(f'episode : {episode}')
    traci.close()


 Retrying in 1 seconds


episode : 0
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 11 BUF 0)                    
 Retrying in 1 seconds


episode : 1 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 31 BUF 0)                    
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 4 BUF 0)                     
 Retrying in 1 seconds


episode : 2 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 31 ACT 31 BUF 1)                    
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 33 ACT 0 BUF 0)                     
 Retrying in 1 seconds


episode : 3 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 30 BUF 0)                    
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 6 BUF 0)                     
 Retrying in 1 seconds


...

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 0.13378009 -0.4766109   0.12567765  0.9279666 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [1.3897921 0.3019121 1.5772308 2.173359 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [1.3295687 0.5188682 1.834615  1.5737872]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 0.01450789 -0.25669134 -0.2026341   0.19069016]
episode : 297
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 1 BUF 0)                     
 Retrying in 1 seconds


state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-2.3894348 -0.8638576 -2.0393913 -2.3830054]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.6477252  -0.94193166 -1.9391372  -2.0075078 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.2042079 -0.6030992 -1.296505  -2.1016798]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.7053417e+00 -9.3953037e-01  5.3286552e-05 -2.5440460e-01]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 1.2990468  -0.52051544  0.17251009  2.790754  ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0]         
Q_values : [-1.0638844  -0.29983598  2.7858531   1.4290475 ]


state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 0.5351707  -0.6873642   0.14880216  1.4872913 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [0.98042554 0.9176453  0.06689155 0.6509105 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.0317982  -0.74716055 -0.14114076 -1.8713989 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.0342842  -0.38997486 -0.7508289  -1.3021526 ]
episode : 298
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 0 BUF 0)                     
 Retrying in 1 seconds


state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.6095005  -0.9043832  -0.58047444 -1.1609628 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.02493275 -1.0724688   0.09783179 -1.190545  ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.7921784  -1.1337869  -0.10005271 -0.9130023 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-2.3561416  -2.1491854  -0.47531825  0.1046021 ]


state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.8433056  -0.7055868  -0.19174445 -0.11923856]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 4 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [2.7270622 2.478274  5.1054955 1.9543148]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [1.7742434 1.1024678 0.788449  1.5416753]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [0.5391774 1.0664245 1.1371824 0.1340875]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-0.42599875  1.1556641   0.34013313  1.1168708 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [ 0.9154787   0.5096234  -0.03679788  0.8940163 ]
episode : 299
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 33 ACT 1 BUF 0)                     


In [66]:
sumoBinary = "/Users/arseneclaustre/sumo/bin/sumo-gui"
sumoBinary2 = "/Users/arseneclaustre/sumo/bin/sumo"
sumoConfig = r"C:/Program Files/rl_project/2025-02-12-16-24-37/osm.sumocfg"
sumoConfig2 = r"C:/Program Files/rl_project/2025-02-25-19-05-44/osm.sumocfg"
# sumoConfig3 = r"/home/psels/code/psels/RL_traffic/RL_traffic/Traditional_traffic/traditional_traffic.sumo.cfg"
sumoConfig3 = r"Traditional_traffic/traditional_traffic.sumo.cfg"


rewards = []
best_score = 0
reward = 0
total_reward = 0
list_values = []
weights = []
wait_times = []
# replay_buffer = deque(maxlen=2000)


sumoCmd = [sumoBinary, "-c", sumoConfig3, "--start"]

if traci.isLoaded():
    traci.close()
traci.start(sumoCmd)
lane_ids =  traci.lane.getIDList()
# print(lane_ids[0])

trafic_light_ids = traci.trafficlight.getIDList()

state = np.array(queue(lane_ids))
action=-1
# print(state)
wait_times.append(0)
for step in range(10000): ## TO CHANGED
    #epsilon = max(1 - episode / 10, 0.01)
    nom_du_feu= traci.trafficlight.getIDList()[0]

    if step%300 == 0:
        state=np.array(queue(lane_ids))
        action = epsilon_greedy_policy(state,0)*2
        # action = np.random.randint(8)
        print("action", action)
        print(traci.trafficlight.getAllProgramLogics(nom_du_feu))
        print(traci.trafficlight.getAllProgramLogics(nom_du_feu)[0].phases[action])
        traci.trafficlight.setPhase(trafic_light_ids[0],action)
    traci.simulationStep()

traci.close()


 Retrying in 1 seconds
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.829101  -1.878626  -1.4764578 -1.3621209]
action 6
(Logic(programID='0', type=0, currentPhaseIndex=0, phases=(Phase(duration=30.0, state='GGrGrrGGrGrr', minDur=30.0, maxDur=30.0), Phase(duration=5.0, state='gyrgrrgyrgrr', minDur=5.0, maxDur=5.0), Phase(duration=20.0, state='grGgrrgrGgrr', minDur=20.0, maxDur=20.0), Phase(duration=5.0, state='grygrrgrygrr', minDur=5.0, maxDur=5.0), Phase(duration=30.0, state='GrrGGrGrrGGr', minDur=30.0, maxDur=30.0), Phase(duration=5.0, state='grrgyrgrrgyr', minDur=5.0, maxDur=5.0), Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0), Phase(duration=5.0, state='grrgrygrrgry', minDur=5.0, maxDur=5.0)), subParameter={}),)
Phase(duration=20.0, state='grrgrGgrrgrG', minDur=20.0, maxDur=20.0)
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-1.829101  -1.878626  -1.4764578 -1.362

In [53]:
print(replay_buffer)

deque([(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), -1, np.int64(0), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 1, np.int64(0), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 3, np.int64(0), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 2, np.int64(-3), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 